# Building Differentiation Instruments Example

In [1]:
import pyblp
import pandas as pd

pyblp.__version__

'0.7.0'

In this example, we'll load the automobile product data from :ref:`references:Berry, Levinsohn, and Pakes (1995)` and build some very simple excluded demand-side instruments for the problem in the spirit of :ref:`references:Gandhi and Houde (2017)`.

In [2]:
formulation = pyblp.Formulation('0 + hpwt + air + mpd + space')
formulation

hpwt + air + mpd + space

In [3]:
product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
product_data.head()

,market_ids,clustering_ids,car_ids,firm_ids,region,shares,prices,hpwt,air,mpd,...,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,supply_instruments0,supply_instruments1,supply_instruments2,supply_instruments3,supply_instruments4,supply_instruments5
0,1971,AMGREM71,129,15,US,0.001051,4.935802,0.528997,0,1.888146,...,0.566217,0.365328,0.659480,0.141017,-0.011161,1.478879,-0.546875,-0.163302,-0.833091,0.301411
1,1971,AMHORN71,130,15,US,0.000670,5.516049,0.494324,0,1.935989,...,0.566217,0.290959,0.173552,0.128205,-0.079317,1.088327,-0.546875,-0.095609,-0.390314,0.289947
2,1971,AMJAVL71,132,15,US,0.000341,7.108642,0.467613,0,1.716799,...,0.566217,0.599771,-0.546387,0.002634,0.021034,0.609213,-0.546875,-0.449818,0.400461,0.434632
3,1971,AMMATA71,134,15,US,0.000522,6.839506,0.426540,0,1.687871,...,0.566217,0.620544,-1.122968,0.089023,-0.090014,0.207461,-0.546875,-0.454159,0.934641,0.331099
4,1971,AMAMBS71,136,15,US,0.000442,8.928395,0.452489,0,1.504286,...,0.566217,0.877198,-1.258575,-0.153840,0.038013,0.385211,-0.546875,-0.728959,1.146654,0.520555


Note that we're excluding the constant column because it yields collinear constant columns of differentiation instruments.

We'll first build "local" differentiation instruments, which are constructed by default, and which consist of counts of "close" rival and non-rival products in each market.

In [4]:
local_instruments = pyblp.build_differentiation_instruments(
    formulation, 
    product_data
)
local_instruments.shape

(2217, 8)

Next, we'll build a more continuous "quadratic" version of the instruments, which consist of sums over squared differences between rival and non-rival products in each market.

In [5]:
quadratic_instruments = pyblp.build_differentiation_instruments(
    formulation, 
    product_data,
    version='quadratic'
)
quadratic_instruments.shape

(2217, 8)

We could also use `interact=True` to include interaction terms in either version of instruments, which would help capture covariances between different product characteristics.